In [ ]:
import requests
import pandas as pd
import os

# Defina sua chave API do FRED
api_key = '1bcbbf6cffa7aa308484d98a6078c54d'

# Defina o caminho do arquivo CSV de entrada
input_csv_path = 'C:/Users/DXX7/PETROBRAS/Integração de Dados - SUB ORC - Documentos/INDICES MACROECONOMICOS/Scripts/Scripts/PROJECAO/Dados de entrada/FRED_projecao_entrada.csv'

# Verificar se o arquivo CSV existe
if not os.path.exists(input_csv_path):
    print(f'Arquivo CSV não encontrado em: {input_csv_path}')
    exit()

# Carregar o arquivo CSV para um DataFrame, especificando o separador como ponto e vírgula e o encoding como latin1
print(f'Lendo arquivo CSV de entrada: {input_csv_path}')
df_indicadores = pd.read_csv(input_csv_path, sep=';', encoding='latin1')

# URL base da API do FRED
base_url = 'https://api.stlouisfed.org/fred/series/observations'

# Função para buscar metadados do FRED
def fetch_metadata(series_id):
    url = f'https://api.stlouisfed.org/fred/series?series_id={series_id}&api_key={api_key}&file_type=json'
    print(f'Fetching metadata from: {url}')  # Imprimir URL para verificação
    response = requests.get(url)
    print(f'Status Code: {response.status_code}')  # Imprimir status da resposta
    if response.status_code == 200:
        try:
            metadata = response.json()
            return metadata
        except ValueError as e:
            print(f'Error decoding JSON for {series_id}: {e}. Response content: {response.text}')  # Imprimir conteúdo da resposta em caso de erro
    else:
        print(f'Failed to fetch metadata for {series_id}. Status Code: {response.status_code}')
    return None

# Função para buscar dados do FRED
def fetch_data(series_id):
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json&observation_start=2000-01-01'
    print(f'Fetching data from: {url}')  # Imprimir URL para verificação
    response = requests.get(url)
    print(f'Status Code: {response.status_code}')  # Imprimir status da resposta
    if response.status_code == 200:
        try:
            data = response.json()
            
            # Buscar metadados da série para obter o nome completo do indicador
            metadata_url = f'https://api.stlouisfed.org/fred/series?series_id={series_id}&api_key={api_key}&file_type=json'
            print(f'Fetching metadata from: {metadata_url}')  # Imprimir URL para verificação
            metadata_response = requests.get(metadata_url)
            print(f'Metadata Status Code: {metadata_response.status_code}')  # Imprimir status da resposta
            
            if metadata_response.status_code == 200:
                metadata = metadata_response.json()
                if 'seriess' in metadata and len(metadata['seriess']) > 0:
                    name = metadata['seriess'][0].get('title', '')  # Obtendo o nome completo do indicador
                    unit = metadata['seriess'][0].get('units', '')
                    frequency = metadata['seriess'][0].get('frequency', '')
                    source = metadata['seriess'][0].get('source', '')
                    api = metadata['seriess'][0].get('api', '')
                    
                    data['name'] = name
                    data['unit'] = unit
                    data['frequency'] = frequency
                    data['source'] = source
                    data['api'] = api
                    
                    return data
                else:
                    print(f'Failed to fetch metadata for {series_id}. Metadata response: {metadata_response.text}')
            else:
                print(f'Failed to fetch metadata for {series_id}. Metadata Status Code: {metadata_response.status_code}')
        except ValueError as e:
            print(f'Error decoding JSON for {series_id}: {e}. Response content: {response.text}')  # Imprimir conteúdo da resposta em caso de erro
    else:
        print(f'Failed to fetch data for {series_id}. Status Code: {response.status_code}')
    return None

# Buscar dados para cada indicador listado no arquivo CSV
data_frames = []

for index, row in df_indicadores.iterrows():
    code = row['codigo']
    
    # Buscar dados do FRED para o indicador atual
    print(f'Buscando dados para o indicador: {code}')
    data = fetch_data(code)
    
    if data and 'observations' in data:
        df = pd.DataFrame(data['observations'])
        
        # Adicionar metadados ao DataFrame
        df['codigo'] = code
        df['nome_completo'] = data.get('name', '')  # Adicionar o nome completo do indicador
        
        df['periodicidade'] = data.get('frequency', '')
        df['unidade'] = data.get('unit', '')
        
        data_frames.append(df)
    else:
        print(f'Nenhum dado encontrado para o indicador: {code}')

# Verificar se algum dado foi coletado
if data_frames:
    # Concatenar todos os DataFrames
    final_df = pd.concat(data_frames, ignore_index=True)
    
    # Remover a coluna 'realtime_end' se existir
    if 'realtime_end' in final_df.columns:
        final_df.drop(columns=['realtime_end'], inplace=True)
    
    # Substituir "." por "," nas colunas numéricas e formatar unidades
    def format_decimal(value):
        try:
            return str(value).replace('.', ',') if pd.notnull(value) else value
        except Exception as e:
            print(f'Error formatting value {value}: {e}')
            return value
    
    final_df = final_df.applymap(format_decimal)
    
    # Remover linhas onde todas as colunas são apenas ","
    final_df = final_df.dropna(how='all', subset=final_df.columns[:-4], inplace=False)
    
    # Excluir linhas que contenham apenas ","
    final_df = final_df[~final_df.apply(lambda row: row.astype(str).str.strip().eq(',').any(), axis=1)]
    
    
    # Salvar taxas em um CSV separado
    taxas_csv_path = 'C:/Users/DXX7/PETROBRAS/Integração de Dados - SUB ORC - Documentos/INDICES MACROECONOMICOS/Scripts/Scripts/PROJECAO/Dados de saida/FRED_projecao_saida.csv'
    final_df.to_csv(taxas_csv_path, index=False, sep=';')  # Salvar usando ponto e vírgula como separador
    print(f'Taxas dos indicadores salvas em: {taxas_csv_path}')
else:
    print('Nenhum dado foi coletado.')


Lendo arquivo CSV de entrada: C:/Users/FNEQ/PETROBRAS/Integração de Dados - SUB ORC - Scripts/Scripts/PROJECAO/Dados de entrada/FRED_projecao_entrada.csv
Buscando dados para o indicador: FEDTARCTH
Fetching data from: https://api.stlouisfed.org/fred/series/observations?series_id=FEDTARCTH&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json&observation_start=2000-01-01
Status Code: 200
Fetching metadata from: https://api.stlouisfed.org/fred/series?series_id=FEDTARCTH&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json
Metadata Status Code: 200
Buscando dados para o indicador: FEDTARCTL
Fetching data from: https://api.stlouisfed.org/fred/series/observations?series_id=FEDTARCTL&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json&observation_start=2000-01-01
Status Code: 200
Fetching metadata from: https://api.stlouisfed.org/fred/series?series_id=FEDTARCTL&api_key=1bcbbf6cffa7aa308484d98a6078c54d&file_type=json
Metadata Status Code: 200
Buscando dados para o indicador: FEDTARCTM

C:\Users\FNEQ\AppData\Local\Temp\ipykernel_11324\2345943408.py:122: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df = final_df.applymap(format_decimal)
